In [5]:
# Imports
import locale
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import win32com.client as win32

# Inicializa o navegador usando o selenium e faz com que ele rode em segundo plano
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
driver = webdriver.Chrome(options=chrome_options)

# Importar e visualizar a base de dados
products_table = pd.read_excel("buscas.xlsx")

In [6]:
# Funções
def data_treatment(product, banned):
    
    # Tratamento dos dados
    product = product.lower()
    product_terms = product.split(' ')
    banned = banned.lower()
    banned_terms = banned.split(' ')

    return product_terms, banned_terms

def google_shopping_search(driver, product, banned, min_price, max_price):
    
    # Retorna os dados tratados
    product_terms, banned_terms = data_treatment(product, banned)
    
    # Acessa o site do google
    driver.get("https://www.google.com/")

    # Acessa o campo de busca e faz a pesquisa
    driver.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(product, Keys.ENTER)

    # Acha e clica no elemento 'Shopping' do menu do google
    elements = driver.find_elements(By.CLASS_NAME, 'hdtb-mitem')
    for element in elements:
        if "Shopping" in element.text:
            element.click()
            break

    # Pega cada resultado (ou div) do google shopping
    matches = driver.find_elements(By.CLASS_NAME, 'sh-dgr__grid-result')
    matches_list = []

    # Pega o preço, nome e link de cada um dos resultados
    for match in matches:
        try:
            price = match.find_element(By.CLASS_NAME, 'a8Pemb').text
            name = match.find_element(By.CLASS_NAME, 'Xjkr3b').text
            link = match.find_element(By.TAG_NAME, 'a').get_attribute('href')
            
            verified = verify_values(price, name, link, product_terms, banned_terms, min_price, max_price)
            if verified: 
                matches_list.append(verified)
        except:
            continue
    # Retorna uma lista de tuplas pra cada resultado
    return matches_list

def buscape_search(driver, product, banned, min_price, max_price):
    
    # Retorna os dados tratados
    product_terms, banned_terms = data_treatment(product, banned)
    
    # Acessa o site do buscapé
    driver.get("https://www.buscape.com.br/")

    driver.find_element(By.XPATH, '//*[@id="new-header"]/div[1]/div/div/div[3]/div/div/div[1]/input').send_keys(product, Keys.ENTER)
    
    time.sleep(10)

    matches = driver.find_elements(By.CLASS_NAME, 'Cell_Content__fT5st')
    matches_list = []

    for match in matches:
        try:
            price = match.find_element(By.CLASS_NAME, 'CellPrice_MainValue__JXsj_').text
            name = match.get_attribute('title')
            link = match.get_attribute('href')

            verified = verify_values(price, name, link, product_terms, banned_terms, min_price, max_price)
            if verified: 
                matches_list.append(verified)
        except:
            continue

    # Retorna uma lista de tuplas pra cada resultado
    return matches_list

def verify_values(price, name, link, product_terms, banned_terms, min_price, max_price):
    
    # Garante que não contenha valores vazios
    if price != '' and name != '' and link != '':
        
        # Trata os preços de string pra int usando o módulo 'locale'
        value = price
        value = value[2:]
        locale.setlocale(locale.LC_ALL, 'pt_BR.utf-8')
        value = locale.atof(value)
        value = float(value)
        
        ban = False

        # Verifica se o produto encontrado contém algum dos termos banidos
        for term in banned_terms:
            if term in name.lower():
                ban = True
                break

        if ban == False:
            # Verifica se o produto encontrado possui as características necessárias
            for term in product_terms:
                if not term in name.lower():
                    ban = True
                    break
            if min_price <= value <= max_price and ban == False:
                    return ((name, price, link))


In [7]:
global_matches_df = pd.DataFrame()
for linha in products_table.index:
    product = products_table.loc[linha, 'Nome']
    banned = products_table.loc[linha, 'Termos banidos']
    min_price = products_table.loc[linha, 'Preço mínimo']
    max_price = products_table.loc[linha, 'Preço máximo']
    
    # Pega o resultado das buscas do google shopping e verifica se algum produto foi encontrado
    google_matches = google_shopping_search(driver, product, banned, min_price, max_price)
    if google_matches:
        google_matches_df = pd.DataFrame(google_matches, columns = ['Produto', 'Preço', 'Link'])
        global_matches_df = global_matches_df.append(google_matches_df)
    
    # Pega o resultado das buscas do buscapé e verifica se algum produto foi encontrado
    buscape_matches = buscape_search(driver, product, banned, min_price, max_price)
    if buscape_matches:
        buscape_matches_df = pd.DataFrame(buscape_matches, columns = ['Produto', 'Preço', 'Link'])
        global_matches_df = global_matches_df.append(google_matches_df)
    

In [8]:
# Cria o arquivo com todos os produtos encontrados
global_matches_df.reset_index(drop = True)
global_matches_df.to_excel('Product_matches.xlsx', index = False)

In [9]:
# Manda o email com todos os produtos encontrados
if len(global_matches_df) > 0:
    outlook = win32.Dispatch('outlook.application')
    mail = outlook.CreateItem(0)
    mail.To = 'destinatario@gmail.com'
    mail.Subject = 'Produto(s) encontrado(s) com o preço desejado!'
    mail.HTMLBody = f'''
    <p>Bom dia, esses foram os produtos encontrados de acordo com o preço e características desejados.</p>
    {global_matches_df.to_html(index = False)}
    Att. Erick, FullStack Web Developer
    '''
    mail.Send()

# Encerra o navegador
driver.quit()